# Hate Speach Detection
I want to build an algorithim to detect hate speach.  This will use the Hate Speach and Offensive Language dataset at https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset?select=labeled_data.csv.  This dataset is 24782 tweets long with users voting on if the tweet is offensive, hate_speach or neither, with the winning label being selected as a class.
## Approach
I will approach this as a text classification problem similar to the BBC problem.  The big difference is that my text can be labeled as either hate speach, or offensive language.  Users were allowed to vote on which category the data fell into but were only permitted to select 1 category when multiple may apply.

I'm going to move the count to a boolean selection.  If either hate speach or offensive language have more than 1/3 of the count, it will be 1, otherwise it will be 0.  After I perform NMF, I will use various models to bucket the new text.  For classification, I will generate an expected value and test various cutoffs for classification.

### Disclaimer
This assignment necissarly contains hate speach, and offensive langauge.  This does not represent the views of the author.

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer as TFIDF

In [2]:
#Initial Import
raw_data = pd.read_csv('C:\\Users\\isaac\\OneDrive\\School\\'\
           'MS\\Unstructured Learning\\Week 5\\'\
           'Unsupervised-Learning-Final\\Data\\'\
           'labeled_data.csv', header = 0)
print(raw_data)

       Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0               0      3            0                   0        3      2   
1               1      3            0                   3        0      1   
2               2      3            0                   3        0      1   
3               3      3            0                   2        1      1   
4               4      6            0                   6        0      1   
...           ...    ...          ...                 ...      ...    ...   
24778       25291      3            0                   2        1      1   
24779       25292      3            0                   1        2      2   
24780       25294      3            0                   3        0      1   
24781       25295      6            0                   6        0      1   
24782       25296      3            0                   0        3      2   

                                                   tweet  
0      !!! RT @m

## Import and Cleaning
### Import
This data imports fairly easily.  I donloaded a copy of the file from kaggle and mirrored it to my GitHub so I could autodownload it for your use.
### Cleaning
#### **Tweets with newline**
There appears to be a mismatch between the number of lines generated and the number of lines imported.  This is caused by newline characters, commas, quotation marks, and other supidity mixed in.  I chose to build my own parser to deal with all the issues and allow me to deal with the various problems with the data.\
**Update**\
There are skipped tweet numbers with no indicator in the dataset other than looking.
##### **Some examples of problematic formating.**
22422,3,0,3,0,1,"This bitch fell straight through the chair... Im high, of course I laughed..\
\
\
\
Even if I wasn't I would've still laughed."

256,3,0,3,0,1,"""@TheNewSAT: #NewSATQuestions\
Yeah bitch, yeah bitch, call me _______:\
a.) Maybe\
b.) Steve-O\
c.) Later\
d.) Jesse Pinkman""\
@machinegunkelly"


#### **Username Cleanup**
I chose to just drop all usernames.  This is due to usernames being random characters and it's out of scope for this project to try to do abusive username detection.
#### **Unicode Characters, numbers, and non-text characters**
I also chose to drop all unicode characters, numbers and most non-text characters.  The exception was the "*" character.  The * character is used to mask offensive words and when used, the resulting word length is fairly standard.

In [3]:
def text_cleanup(text):
    #Remove usernames replace @xxx: with whitespace
    text = re.sub("@.*?:", " ", text)
    #remove unicode characters
    text = re.sub("&#[0-9]{6};", " ", text)
    #lowercase all text
    text = text.lower()
    #remove URL's
    text = re.sub('http[s]?:\/\/.*?"', " ", text)
    #remove special characters and numberrs'
    #I chose to leave in *'s as they are a common manipulation to get around filters
    text = re.sub("[^a-z\*]", " ", text)
    #remove repeat whitespace and newlines
    text = re.sub("\s\s+", " ", text)
    text = re.sub("\n+", "", text)
    return text
raw_data["tweet"] = raw_data["tweet"].apply(text_cleanup)
print(raw_data)

        


       Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0               0      3            0                   0        3      2   
1               1      3            0                   3        0      1   
2               2      3            0                   3        0      1   
3               3      3            0                   2        1      1   
4               4      6            0                   6        0      1   
...           ...    ...          ...                 ...      ...    ...   
24778       25291      3            0                   2        1      1   
24779       25292      3            0                   1        2      2   
24780       25294      3            0                   3        0      1   
24781       25295      6            0                   6        0      1   
24782       25296      3            0                   0        3      2   

                                                   tweet  
0       rt as a 

In [4]:
#Find why I still only have 24783 coumns when the dataset says it has 25.3k
for i in range(0,1000):
    if int(raw_data['tweet num'][i]) != i:
        print (i)
        break
    

KeyError: 'tweet num'

## EDA Procedure
### Formatting Data
I started with formatting my data so that hate speach, offensive language values are 1 if they have at least 1/3 of the votes, otherwise they will be 0.
### Bagging
I chose to use TF-IDF for my bagging procedure.  I will do my testing while removing common english stopwords.  



In [ ]:
#TF IDF
